Libraries

In [9]:
import pandas as pd
import numpy as np

import requests
from bs4 import BeautifulSoup

import re

Słowniki do przerzucenia do utilsów

In [89]:
ligi = {
    'ekstraklasa':'https://ligafanow.pl/rozgrywki/tabela/30/235',
    '1liga':'https://ligafanow.pl/rozgrywki/tabela/30/234',
    '2liga':'https://ligafanow.pl/rozgrywki/tabela/30/231',
    '3liga':'https://ligafanow.pl/rozgrywki/tabela/30/232',
    '4liga':'https://ligafanow.pl/rozgrywki/tabela/30/233',
    '5liga':'https://ligafanow.pl/rozgrywki/tabela/30/230',
    '6liga':'https://ligafanow.pl/rozgrywki/tabela/30/229',
    '7liga':'https://ligafanow.pl/rozgrywki/tabela/30/228',
    '8liga':'https://ligafanow.pl/rozgrywki/tabela/30/227',
    '9liga':'https://ligafanow.pl/rozgrywki/tabela/30/226',
    '10liga':'https://ligafanow.pl/rozgrywki/tabela/30/225',
    '11liga':'https://ligafanow.pl/rozgrywki/tabela/30/224',
    '12liga':'https://ligafanow.pl/rozgrywki/tabela/30/223',
    '13liga':'https://ligafanow.pl/rozgrywki/tabela/30/236'
}
ligi_strzelcy = {key: value.replace('tabela', 'strzelcy') + '?loadpl=all' for key, value in ligi.items()}

Funkcje do przerzucenia do utilsów

Funkcje
- get_table() zwraca tabelę podanej ligi
- get_matches(liga, kolejka, drużyna) zwraca wyniki w danej lidze
- get_scorers(liga, drużyna)zwraca najlepszych strzelców danej ligi /zespołu
- get_team_details() zwraca listę zawodników danego zespołu
- get_match_events() zwraca dokładny przebieg danego meczu

In [36]:
df = get_table('2liga')

In [86]:
# Main functions
def get_table(league, ligi = ligi):
    """
    Retrieves and parses the league table data from the provided league name using the corresponding URL.
    
    Parameters:
    - league (str): The name of the league for which the table data will be fetched.
    - ligi (dict): A dictionary containing league names as keys and their corresponding URLs as values.
    
    Returns:
    - pandas.DataFrame: A DataFrame representing the league table, with headers and data extracted from the HTML
      content of the league's URL. The DataFrame is cleaned and formatted for further analysis.
    """
    # Get response from url
    response = requests.get(ligi[league])
    soup = BeautifulSoup(response.text, 'html.parser')
    
    # Find table in url
    table = soup.find('table')
    
    # Empty lists for data
    headers = []
    rows = []
    
    # Get headers
    for th in table.findAll('th'):
        headers.append(th.text.strip())
        
    # Get data
    for tr in table.findAll('tr'):
        row = []
        for td in tr.findAll(['td', 'th']):
            row.append(td.text.strip())
        rows.append(row)
        
    # Create dataframe
    df = pd.DataFrame(rows, columns=headers)
    df.columns = df.iloc[0]
    df = df[1:]
    df = convert_to_int(df)
    df = adjust_dataframe(df)
    
    return df

def get_matches(league, round_=None, team=None, ligi=ligi):
    """
    Retrieves and compiles the match data for a given league, with optional filters for specific rounds or teams.

    Parameters:
    - league (str): The name of the league for which match data will be fetched.
    - round_ (int or None): If provided, filters the match data to include only matches from the specified round.
    - team (str or None): If provided, filters the match data to include only matches involving the specified team.
    - ligi (dict): A dictionary containing league names as keys and their corresponding URLs as values.

    Returns:
    - pandas.DataFrame: A DataFrame representing the match data for the specified league. The DataFrame is cleaned
      and formatted for further analysis. Optional filters based on round or team are applied if provided.
    """
    links = table_of_links(league)
    wyniki = []
    for zespol in links['Zespół']:
        wynik = take_table_results(extract_mecze_links(zespol))
        # Przykład użycia
        wynik['Kol.'] = wynik['Kol.'].apply(extract_round)   
        wyniki.append(wynik)
    df = pd.concat(wyniki)
    df = df.drop_duplicates()
    df = convert_to_int(df)
    df.sort_values(['Kol.', 'Godz.'], ascending=[False, True], inplace=True)  # Dodano inplace=True
    
    # When round added
    if round_ is not None:
        df = df[df['Kol.'] == round_]
    
    # When team added
    if team is not None:
        df = df[(df['Gospodarz'] == team) | (df['Gość'] == team)]      
    
    return df

In [87]:
df = get_matches('2liga', round_=7)

In [88]:
df

,Kol.,Data,Godz.,Gospodarz,,Gość,Video,,Sędzia,Boisko,Raport
2,7,2023-11-12,09:00,Dziki Młochów,7 : 4,Tylko Zwycięstwo,,,Volodymyr Molokov,Arena AWF,Raport
2,7,2023-11-12,14:00,Orzeły Stolicy,3 : 5,Husaria Mokotów,,,Volodymyr Molokov,Arena AWF,Raport
2,7,2023-11-12,17:00,Korsarze,3 : 8,AFC Goodfellas,,,Volodymyr Molokov,Arena AWF,Raport
2,7,2023-11-12,19:00,UEFA Mafia Ursynów,8 : 2,Warszawska Ferajna,,,Piotr Krajczyński,Arena AWF,Raport
2,7,2023-11-12,19:00,Green Lantern,6 : 18,KSB Warszawa,,,Rafał Szczytniewski,Arena AWF,Raport


In [83]:
df[df['Kol.']==9]

,Kol.,Data,Godz.,Gospodarz,,Gość,Video,,Sędzia,Boisko,Raport
0,9,2023-11-26,11:00,Tylko Zwycięstwo,10 : 2,Warszawska Ferajna,,,Łukasz Matlęga,Arena Grenady,Raport
0,9,2023-11-26,19:00,Green Lantern,4 : 7,UEFA Mafia Ursynów,,,Maciej Majewski,Arena AWF,Raport
0,9,2023-11-26,20:00,Korsarze,0 : 3,Orzeły Stolicy,,,Rafał Szczytniewski,Arena AWF,Raport
0,9,2023-11-26,17:00,AFC Goodfellas,6 : 3,Dziki Młochów,,,Łukasz Matlęga,Arena Grenady,Raport
0,9,2023-11-26,21:00,KSB Warszawa,3 : 8,Husaria Mokotów,,,Konrad Gawroński,Arena AWF,Raport


In [43]:
# Aside functions

def table_of_links(league, ligi = ligi):
    """
    Function that extracts a table with columns containing links from the given URL
    """
    # Get response from URL
    response = requests.get(ligi[league])
    soup = BeautifulSoup(response.text, 'html.parser')

    # Find the table in the URL
    table = soup.find('table')

    # Empty lists for data
    headers = []
    link_columns = []

    # Get headers
    for th in table.findAll('th'):
        headers.append(th.text.strip())

    # Get data
    for tr in table.findAll('tr'):
        row = []
        for td in tr.findAll(['td', 'th']):
            # Check if the td tag contains an 'a' tag (link)
            link = td.find('a')
            if link:
                # If there is a link, append the link's href attribute
                row.append(link.get('href'))
            else:
                # If there is no link, append an empty string
                row.append('')
        link_columns.append(row)

    # Create a DataFrame with link columns
    df = pd.DataFrame(link_columns, columns=headers)

    # Remove rows with all empty values
    df = df.replace('', pd.NA).dropna(how='all')

    # Remove columns with all empty values
    df = df.replace('', pd.NA).dropna(axis=1, how='all')
    
    df = df.reset_index(drop=True)

    return df

def triple_strings(list_):
    """
    Formats data from league tables by grouping every three consecutive elements with varying space separations into
    clear rows. This function is designed to enhance the readability of league table data.

    Parameters:
    - list_ (list): A list containing strings, typically extracted from league tables, where data elements are
      separated by different numbers of spaces.

    Returns:
    - list of str: A list of formatted strings, with every three consecutive elements joined into clear rows for
      improved presentation of league table data.
    """
    result = []
    for i in range(0, len(list_), 3):
        triple = list_[i:i+3]
        result.append(' '.join(triple))
    return result

def string_divide(string, min_space_number=2):
    """
    Splits a given string into substrings based on consecutive spaces, while allowing for a minimum number of spaces
    between substrings. The function aims to identify meaningful segments of the input string, considering a specified
    minimum space count to distinguish between substrings.

    Parameters:
    - string (str): The input string to be divided.
    - min_space_number (int): The minimum number of consecutive spaces required to separate substrings. Default is 2.

    Returns:
    - list of str: A list containing the identified substrings from the input string, excluding leading and trailing
      whitespaces. Empty substrings are excluded from the result.
    """
    substrings = []
    current_podstring = ""
    space_number = 0

    for sign in string:
        if sign == ' ':
            if space_number >= min_space_number-1:
                substrings.append(current_podstring.strip())
                current_podstring = ""
            elif current_podstring.strip():
                current_podstring += ' '
            space_number += 1
        else:
            current_podstring += sign
            space_number = 0

    if current_podstring.strip():
        substrings.append(current_podstring.strip())

    substrings = [substring for substring in substrings if substring.strip()]

    return substrings

def convert_to_int(df):
    """
    Converts columns of a DataFrame to the int type where possible.
    
    Parameters:
    - df (pandas.DataFrame): DataFrame to be converted.
    
    Returns:
    - pandas.DataFrame: Modified DataFrame.
    """
    for column in df.columns:
        try:
            df[column] = df[column].astype(int)
        except ValueError:
            pass  # If conversion is not possible, proceed to the next column
    return df

def divide_events(string):
    pattern = re.compile(r'([PWR]) \d{4}-\d{2}-\d{2} \d{2}:\d{2}')
    # Wyszukiwanie pasujących sekwencji w tekście
    matches = pattern.findall(text)

    # Gromadzenie rodzajów wydarzeń
    event_types = [match[0] for match in matches]

    # Tworzenie stringa z rodzajami wydarzeń, oddzielonymi przecinkami
    result_string = ",".join(event_types)
    
    return result_string

def adjust_dataframe(df):
    df = df.replace('', pd.NA).dropna(axis=1, how='all')
    # Linijka do poprawy jeśli jakieś wartości są uzupełnione
    df.columns = ['Poz', 'Zespół', 'Mecze_rozegrane', 'Pkt.', 'Pkt.', 'Z', 'R', 'P', 'BZ', 'BS', '+/-','Forma']
    df['Forma'] = df['Forma'].apply(lambda x: divide_events(x))
    df = df.sort_values('Poz')
    df = df.reset_index(drop=True)
    return df

def team_form_df(df, team):
    data = df[df['Zespół'] == team]['Forma'][0] 
    
    # Podzielenie danych na wiersze
    rows = re.split(r'([PWR])\s+(\d{4}-\d{2}-\d{2} \d{2}:\d{2})', data)[1:]
    # Columns
    columns = ["Typ", "Data", "Czas", "Drużyna 1", "Wynik", "Drużyna 2", "Arena"]
    team_df = pd.DataFrame([row.split()[0:3] + [string_divide(row)[1]] + [string_divide(row)[2]]+ [string_divide(row)[3]] + [string_divide(row)[4]] for row in triple_strings(rows)], columns=columns)
    return team_df

def extract_mecze_links(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')

    mecze_links = []
    for link in soup.find_all('a', href=True):
        if 'mecze' in link['href']:
            mecze_links.append(link['href'])

    return mecze_links[0]

def extract_mecze_details_links(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')

    mecze_details_links = []
    for link in soup.find_all('a', href=True):
        if 'raport' in link['href'] and 'veo' not in link['href']:
            mecze_details_links.append(link['href'])

    return list(set(mecze_details_links))

def take_table_results(url):
    """
    Function that takes a table of league from the given url
    """
    # Get response from url
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')

    # Find table in url
    table = soup.find('table')

    # Empty lists for data
    headers = []
    rows = []

    # Get headers
    for th in table.findAll('th'):
        headers.append(th.text.strip())

    # Get data
    for tr in table.findAll('tr')[1:]:  # Skip the first row as it contains headers
        row = []

        # Check if the row contains data or nested structure
        if tr.find('td', class_='hideonmobie'):
            # Extract data from nested structure
            mobile_data = tr.find('span', class_='d-block d-sm-none')
            row.append(mobile_data.find('div', class_='text-uppercase').text.strip())  # Extract mobile data
            row.extend([td.text.strip() for td in mobile_data.find_all('div', class_='text-center')])

        else:
            # Extract data from regular structure
            row.extend([td.text.strip() for td in tr.find_all(['td', 'th'])])

        rows.append(row)

    # Create DataFrame
    df = pd.DataFrame(rows, columns=headers)

    return df

# Customowa funkcja do wyciągania wartości
def extract_round(row):
    match = re.search(r'kolejka (\d+)', row)
    if match:
        return match.group(1)
    return None

In [110]:
# Tabela

In [111]:
df = take_table(url)

In [112]:
team_form_df(df, df['Zespół'][0])

,Typ,Data,Czas,Drużyna 1,Wynik,Drużyna 2,Arena
0,P,2023-11-12,22:00,Mareckie Wygi,13 : 4,Hiszpański Galeon,Arena AWF
1,P,2023-11-05,11:00,Hiszpański Galeon,4 : 9,Saska Kępa,Arena AWF
2,P,2023-10-29,17:00,Legion,5 : 1,Hiszpański Galeon,Arena AWF
3,P,2023-11-26,13:00,Hiszpański Galeon,4 : 7,Na2Nóżkę,Arena AWF
4,W,2023-10-15,16:30,Kozice Warszawa,2 : 8,Hiszpański Galeon,Arena Picassa


In [113]:
# Linki do wyników

In [114]:
dff = table_of_links(url)

In [115]:
dff

,Zespół,Forma
0,https://ligafanow.pl/statystyki/druzyna/hiszpa...,https://ligafanow.pl/statystyki/raport/8198
1,https://ligafanow.pl/statystyki/druzyna/saska-...,https://ligafanow.pl/statystyki/raport/8197
2,https://ligafanow.pl/statystyki/druzyna/fc-pol...,https://ligafanow.pl/statystyki/raport/8197
3,https://ligafanow.pl/statystyki/druzyna/shot-dj,https://ligafanow.pl/statystyki/raport/8196
4,https://ligafanow.pl/statystyki/druzyna/mareck...,https://ligafanow.pl/statystyki/raport/8198
5,https://ligafanow.pl/statystyki/druzyna/legionfc,https://ligafanow.pl/statystyki/raport/8199
6,https://ligafanow.pl/statystyki/druzyna/warsaw...,https://ligafanow.pl/statystyki/raport/8196
7,https://ligafanow.pl/statystyki/druzyna/kozice...,https://ligafanow.pl/statystyki/raport/8199
8,https://ligafanow.pl/statystyki/druzyna/na2nk,https://ligafanow.pl/statystyki/raport/8200
9,https://ligafanow.pl/statystyki/druzyna/brd-yo...,https://ligafanow.pl/statystyki/raport/8200


In [11]:
# Pobranie wyników

In [119]:
wynik  = take_table_results(extract_mecze_links(dff.Zespół[0]))
# Przykład użycia
wynik['Kol.'] = wynik['Kol.'].apply(extract_round)
wynik

,Kol.,Data,Godz.,Gospodarz,,Gość,Video,,Sędzia,Boisko,Raport
0,9,2023-11-26,13:00,Hiszpański Galeon,4 : 7,Na2Nóżkę,,,Rafał Szczytniewski,Arena AWF,Raport
1,8,2023-11-19,16:00,Hiszpański Galeon,- : -,FC Polska Górom,,,Łukasz Matlęga,Arena AWF,Raport
2,7,2023-11-12,22:00,Mareckie Wygi,13 : 4,Hiszpański Galeon,,,Piotr Krajczyński,Arena AWF,Raport
3,6,2023-11-05,11:00,Hiszpański Galeon,4 : 9,Saska Kępa,,,Konrad Gawroński,Arena AWF,Raport
4,5,2023-10-29,17:00,Legion,5 : 1,Hiszpański Galeon,,,Piotr Krajczyński,Arena AWF,Raport
5,4,2023-10-15,16:30,Kozice Warszawa,2 : 8,Hiszpański Galeon,,,Piotr Krajczyński,Arena Picassa,Raport
6,3,2023-10-08,17:00,Shot DJ,2 : 2,Hiszpański Galeon,,,Dariusz Oleksiewicz,Arena AWF,Raport
7,2,2023-10-01,18:00,Warsaw Gunners FC,4 : 2,Hiszpański Galeon,,,Piotr Krajczyński,Arena AWF,Raport
8,1,2023-09-24,18:00,Hiszpański Galeon,3 : 6,BRD Young Warriors,,,Artur Bębeński,Arena Grenady,Raport


In [118]:
def reports_links(url):
    list_ = extract_mecze_details_links(extract_mecze_links(url)) #dff.Zespół[0])
    list_ = ['https://ligafanow.pl/'+ elem for elem in list_]
    return list_

In [120]:
b = reports_links(dff.Zespół[0])
b

['https://ligafanow.pl//statystyki/raport/8058',
 'https://ligafanow.pl//statystyki/raport/7989',
 'https://ligafanow.pl//statystyki/raport/7919',
 'https://ligafanow.pl//statystyki/raport/8198',
 'https://ligafanow.pl//statystyki/raport/8272',
 'https://ligafanow.pl//statystyki/raport/7754',
 'https://ligafanow.pl//statystyki/raport/7844',
 'https://ligafanow.pl//statystyki/raport/8130',
 'https://ligafanow.pl//statystyki/raport/7683']

In [121]:
def match_details(path):
    response = requests.get(path)

    soup = BeautifulSoup(response.text, 'html.parser')

    # Znajdź wszystkie tabelki na stronie
    tables = soup.find_all('table')

    # Przechowaj ramki danych w liście
    dataframes = []

    # Sprawdź, czy znaleziono jakiekolwiek tabele
    if tables:
        # Iteruj przez wszystkie znalezione tabele
        for table in tables:
            # Wydziel dane z tabeli do listy słowników
            table_data = []
            rows = table.find_all('tr')

            # Sprawdź, czy istnieje przynajmniej jeden wiersz
            if rows:
                header = [header.text.strip() for header in rows[0].find_all(['th', 'td'])]

                # Iteruj przez pozostałe wiersze
                for row in rows[1:]:
                    row_data = [cell.text.strip() for cell in row.find_all(['th', 'td'])]
                    table_data.append(dict(zip(header, row_data)))

                # Przekształć dane do DataFrame
                df = pd.DataFrame(table_data)
                dataframes.append(df)

    # Wydrukuj ramki danych
    df = pd.concat([dataframes[1],dataframes[2]]).dropna()
    
    return df

In [122]:
match_details(b[0])

,,Imie i nazwisko,Wydarzenie,Ilość
1,,Maksym Popov,Bramka stracona,1
3,,Igor Polskyi,Bramka,2
4,,Illia Kopyl,Bramka,1
5,,Vladyslav Barabash,Bramka,2
7,,Anrdei Benenko,Asysta,2
8,,Illia Kopyl,Asysta,1
9,,Vladyslav Barabash,Asysta,1
11,,Vladyslav Barabash,SuperStar,3
13,,Maksym Popov,6 - kolejki,1
1,,Artur Pręgowski,Bramka stracona,5


In [123]:
# Funkcja do ekstrakcji danych z tabeli
def extract_table_data(table):
    table_data = []
    
    # Znajdź nagłówki z pierwszego wiersza thead
    header_row = table.select('thead tr')[1]  # Use the second row to get headers with tooltip
    headers = []

    for header in header_row.find_all(['th', 'td']):
        if 'tooltip' in header.attrs:
            headers.append(header['tooltip'].strip())
        else:
            headers.append(header.text.strip())

    rows = table.select('tbody tr')
    
    for row in rows:
        player_data = {}
        columns = row.find_all(['td', 'th'])
        
        # Iteruj przez kolumny w danym wierszu
        for i, col in enumerate(columns):
            player_data[headers[i]] = col.text.strip()
        
        table_data.append(player_data)

    return table_data

def extract_team(url):
    response = requests.get(url)
    soup = BeautifulSoup(response1.text, 'html.parser')
    
    table1 = soup.find('table', {'id': 'mytxablecc'})
    table2 = soup.find('table', {'id': 'mytxablec'})
    
    # Sprawdź, czy obie tabele zostały znalezione
    if table1 and table2:
        # Ekstrahuj dane z obu tabel
        data1 = extract_table_data(table1)
        data2 = extract_table_data(table2)
    
    # Połącz dane z obu tabel w jedną listę
    combined_data = data1 + data2
    
    # Przekształć dane do DataFrame
    df = pd.DataFrame(combined_data)
    df.columns = ['Imie i nazwisko', 'Numer', 'Liczba wystepów', 'Liczba bramek',
       'Asysty', 'Kanadyjcztk', 'Superstar', 'Top6', 'MVP', 'Czerwone kartki',
       'Zółte kartki', 'Stracone bramki', 'Samobój', 'Obronione karne',
       'Czyste konto', 'Gold Team','ID']
    return df

In [124]:
extract_team(dff.Zespół[0])

,Imie i nazwisko,Numer,Liczba wystepów,Liczba bramek,Asysty,Kanadyjcztk,Superstar,Top6,MVP,Czerwone kartki,Zółte kartki,Stracone bramki,Samobój,Obronione karne,Czyste konto,Gold Team,ID
0,Jędrzej Skawina,,4,1,0,1,0,0,0,0,0,21,0,0,0,0,8098
1,Artur Pręgowski,,4,0,0,0,0,0,0,0,1,16,0,0,0,0,6970
2,Dominik Sposób,,1,0,0,0,1,0,0,0,0,4,0,0,0,0,9016
3,Janek Grzybowski,,0,0,0,0,0,0,0,0,0,0,0,0,0,0,9015
4,Misha Lishtwan,,4,2,1,3,0,0,0,0,0,0,0,0,0,0,8983
5,Aleksander Popecki,,2,0,2,2,1,0,0,0,0,0,0,0,0,0,8969
6,Piotr Lewit,,0,0,0,0,0,0,0,0,0,0,0,0,0,0,8967
7,Jura Lunio,,5,3,2,5,3,1,0,0,0,0,0,0,0,0,8946
8,Bartosz Krzeszowski,,2,3,0,3,0,0,0,0,0,0,0,0,0,0,8945
9,Piotr Ignatowski,,0,0,0,0,0,0,0,0,0,0,0,0,0,0,8424


In [1]:
# Strzelcy

---------------------------------------------------------------------

In [25]:
# Pola do zrobienia z nich tabel

In [123]:
df.Forma[2]

'W 2023-11-12 21:00    In Plus & Pojemna Halina    5 : 7  Gladiatorzy Eternis    Arena AWF  W 2023-11-05 19:00    Gladiatorzy Eternis   14 : 2    ALPAN     Arena AWFR 2023-10-29 18:00    EXC Mobile Ochota   4 : 4  Gladiatorzy Eternis    Arena AWF  W 2023-11-26 18:00    TUR Ochota    1 : 3  Gladiatorzy Eternis    Arena AWF  W 2023-10-15 20:00    Gladiatorzy Eternis   14 : 4    Esportivo Varsovia     Arena AWF'

In [124]:
# zapisanie zmiennej tekstowej do zrobienia z niej tabeli
data = df.Forma[2]

In [125]:
# Prawdziwy kod

In [126]:
# Podzielenie danych na wiersze
rows = re.split(r'([PWR])\s+(\d{4}-\d{2}-\d{2} \d{2}:\d{2})', data)[1:]

In [127]:
def triple_strings(lista):
    wynik = []
    for i in range(0, len(lista), 3):
        trojka = lista[i:i+3]
        wynik.append(' '.join(trojka))
    return wynik

def rozdziel_na_podteksty(tekst, minimalna_liczba_spacji=2):
    podteksty = []
    aktualny_podtekst = ""
    liczba_spacji = 0

    for znak in tekst:
        if znak == ' ':
            if liczba_spacji >= minimalna_liczba_spacji-1:
                podteksty.append(aktualny_podtekst.strip())
                aktualny_podtekst = ""
            elif aktualny_podtekst.strip():
                aktualny_podtekst += ' '
            liczba_spacji += 1
        else:
            aktualny_podtekst += znak
            liczba_spacji = 0

    if aktualny_podtekst.strip():
        podteksty.append(aktualny_podtekst.strip())

    podteksty = [podtekst for podtekst in podteksty if podtekst.strip()]

    return podteksty

In [129]:

# Utwórz DataFrame
columns = ["Typ", "Data", "Czas", "Drużyna 1", "Wynik", "Drużyna 2", "Arena"]
df = pd.DataFrame([row.split()[0:3] + [rozdziel_na_podteksty(row)[1]] + [rozdziel_na_podteksty(row)[2]]+ [rozdziel_na_podteksty(row)[3]] + [rozdziel_na_podteksty(row)[4]] for row in triple_strings(rows)], columns=columns)

In [131]:
df

,Typ,Data,Czas,Drużyna 1,Wynik,Drużyna 2,Arena
0,W,2023-11-12,21:00,In Plus & Pojemna Halina,5 : 7,Gladiatorzy Eternis,Arena AWF
1,W,2023-11-05,19:00,Gladiatorzy Eternis,14 : 2,ALPAN,Arena AWF
2,R,2023-10-29,18:00,EXC Mobile Ochota,4 : 4,Gladiatorzy Eternis,Arena AWF
3,W,2023-11-26,18:00,TUR Ochota,1 : 3,Gladiatorzy Eternis,Arena AWF
4,W,2023-10-15,20:00,Gladiatorzy Eternis,14 : 4,Esportivo Varsovia,Arena AWF
